In [1]:
train_file = "./data/spooky/train.csv"

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv(train_file)

In [4]:
df

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL
...,...,...,...
19574,id17718,"I could have fancied, while I looked at it, th...",EAP
19575,id08973,The lids clenched themselves together as if in...,EAP
19576,id05267,"Mais il faut agir that is to say, a Frenchman ...",EAP
19577,id17513,"For an item of news like this, it strikes us i...",EAP


In [5]:
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatiser = WordNetLemmatizer()

In [6]:
replacement_chars = {c: ' ' for c in string.punctuation}
replacement_chars[''] = ''

In [7]:
translator = str.maketrans("".join(replacement_chars.keys()), "".join(replacement_chars.values()))

In [8]:
from nltk import ngrams

stops = stopwords.words('english')
def text_process(tex):
    tex = tex.lower()
    tex = tex.translate(translator)
    tex = " ".join(lemmatiser.lemmatize(t, pos="v") for t in tex.split())
    return ngrams([w for w in tex.split() if w not in stops], 1)

In [9]:
# Importing necessary libraries
from sklearn.preprocessing import LabelEncoder
y = df['author']
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

In [10]:
y

array([0, 1, 0, ..., 0, 0, 1])

In [11]:
X = df['text']

In [12]:
import time
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.20, random_state=int(time.time()))

In [13]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(15663,) (3916,) (15663,) (3916,)


In [42]:
# Importing necessary libraries
from sklearn.feature_extraction.text import CountVectorizer
bow_transformer=CountVectorizer(stop_words="english", ngram_range=(1, 2)).fit(X_train)
text_bow_train=bow_transformer.transform(X_train)
text_bow_test=bow_transformer.transform(X_test)

In [43]:
names = bow_transformer.get_feature_names()
print(len(names), names[:5])

174201 ['aaem', 'ab', 'ab te', 'aback', 'aback breeze']


In [44]:
text_bow_train

<15663x174201 sparse matrix of type '<class 'numpy.int64'>'
	with 339167 stored elements in Compressed Sparse Row format>

In [45]:
# Importing necessary libraries
from sklearn.naive_bayes import MultinomialNB# instantiating the model with Multinomial Naive Bayes..
model = MultinomialNB().fit(text_bow_train, y_train)

In [46]:
model.score(text_bow_train, y_train)

0.9851880227287237

In [47]:
model.score(text_bow_test, y_test)

0.8345250255362615

In [48]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True, sublinear_tf=True)

In [49]:
wc_vector_train = text_bow_train

In [50]:
tfidf_fitted_transformer = tfidf_transformer.fit(wc_vector_train)

In [51]:
tfidf_train = tfidf_fitted_transformer.transform(wc_vector_train)

In [52]:
tfidf_train

<15663x174201 sparse matrix of type '<class 'numpy.float64'>'
	with 339167 stored elements in Compressed Sparse Row format>

In [53]:
wc_vector_test = text_bow_test

In [54]:
tfidf_test = tfidf_fitted_transformer.transform(wc_vector_test)

In [55]:
# Importing necessary libraries
from sklearn.naive_bayes import MultinomialNB
model1 = MultinomialNB(alpha=0.1, fit_prior=True).fit(tfidf_train, y_train)

In [56]:
model1.score(tfidf_train, y_train)

0.9961693162229458

In [57]:
model1.score(tfidf_test, y_test)

0.8437180796731358